In [ ]:
import requests
import pandas as pd

api_url = "https://ows.goszakup.gov.kz/v3/refs/ref_countries"
token = "d5c3d78fc111d88a0a37b4ab8f83cbd5"
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}
file = "countries.csv"

def retrieve_countries():
    """Получает справочник стран через API и сохраняет в CSV."""
    countries_list = []
    next_page = None

    while True:
        try:
            params = {"limit": 500}
            if next_page:
                params["page"] = "next"
                params["search_after"] = next_page

            response = requests.get(api_url, headers=headers, params=params, timeout=10)
            response.raise_for_status()
            response_data = response.json()

            records = response_data.get("items", [])
            if not records:
                print("Все данные загружены.")
                break

            for record in records:
                countries_list.append({
                    "code": record.get("code"),
                    "name_kz": record.get("name_kz"),
                    "code_2": record.get("code_2"),
                    "name_ru": record.get("name_ru"),
                    "code_3": record.get("code_3"),
                })

            print(f"Загружено записей: {len(countries_list)}")
            next_page = response_data.get("next_page")
            if not next_page:
                break

        except requests.exceptions.RequestException as err:
            print(f"Ошибка запроса: {err}")
            break

    export_to_csv(countries_list)

def export_to_csv(dataset):
    """Записывает данные в CSV-файл."""
    if not dataset:
        print("Нет данных для экспорта.")
        return

    data_frame = pd.DataFrame(dataset)
    data_frame.to_csv(
        file,
        index=False,
        encoding="utf-8-sig",
        sep="|",
        quotechar="'",
        escapechar="\\"
    )
    print(f"Данные сохранены в {file}")

retrieve_countries()